Task №1. Access settings

```
grant usage on schema public to planadmin;
grant usage on schema public to planmanager;

grant select on all tables in schema public to planadmin;
grant select on all tables in schema public to planmanager;

grant select, insert, update, delete on plan_data to planadmin;
grant select, insert, update, delete on plan_data to planmanager;

grant select, insert, update, delete on plan_status to planadmin;
grant select, update on plan_status to planmanager;

grant select, insert, update, delete on country_managers to planadmin;
grant select on country_managers to planmanager;

grant select, update on v_plan_edit to planmanager;
grant select on v_plan to planmanager;

create user ivan;

grant planadmin to ivan;

create user sophie;
create user kirill;

grant planmanager to sophie;
grant planmanager to kirill;

insert into country_managers (username, country) values 
('sophie', 'US'), ('sophie', 'CA'), ('kirill', 'FR'), ('kirill', 'GB'), ('kirill', 'DE'), ('kirill', 'AU');
```

---

Task №2. product2 & country 2 materialized views

```
# The ‘product2’ view should contain the product and its category.
create materialized view product2 as
	select p3.productcategoryid as pcid, p.productid, p3."name" as pcname, p."name" as pname from product p 
	join productsubcategory p2 on p.productsubcategoryid=p2.productsubcategoryid
	join productcategory p3 on p2.productcategoryid=p3.productcategoryid 
with no data;

refresh materialized view product2;

# The ‘country2’ view should be filled with unique codes of the countries where the shops are located (the type of address is Main Office). 

create materialized view country2 as
	select distinct a.countryregioncode as countrycode from customer c
	join customeraddress c1 on c1.customerid = c.customerid
	join address a on c1.addressid = a.addressid
	where c.companyname is not null and c.lastname is null
	and c1.addresstype='Main Office'
with no data;

refresh materialized view country2;

# Allow managers and administrators to read from these views
grant select on country2 to planadmin;
grant select on country2 to planmanager;

```

---

Task №3. Loading data into the company table

```
insert into company (cname, countrycode, city)
select c.companyname as cname, a.countryregioncode as countrycode, a.city as city
from customer c
join customeraddress c1 on c.customerid = c1.customerid 
join address a on c1.addressid = a.addressid 
where c.companyname is not null and c.lastname is null
and c1.addresstype ='Main Office';
```
---

Task №4. Company classification by annual amount of orders

```
with yearly_sum as (
    select sum(s.subtotal) as yearsum, date_part('y', s.orderdate) as "year" 
    from customer c 
    inner join company c2 on c.companyname = c2.cname
    join salesorderheader s on c.customerid = s.customerid
    where date_part('y', s.orderdate) in (2012, 2013)
    group by "year"),
company_yearly_sales as (
    select c2.id, sum(s.subtotal) as salestotal, date_part('y', s.orderdate) as "year" 
    from customer c 
    inner join company c2 on c.companyname = c2.cname
    join salesorderheader s on c.customerid = s.customerid
    where date_part('y', s.orderdate) in (2012, 2013)
    group by c2.id, "year"
    order by "year" asc, salestotal desc),
company_running_total as (
    select *, sum(salestotal) over(partition by "year" order by salestotal desc) as srt from company_yearly_sales)
insert into company_abc
select id as cid, 
    salestotal, 
    case when srt <= (yearsum * 0.8) then 'A'
    when srt <= (yearsum * 0.95) then 'B'
    else 'C' end as cls,
    t1."year"
from company_running_total as t1
left join yearly_sum as t2 on t1."year"=t2."year";
```
---

Task №5. Finding quarterly sales amount by company, and
product category

```

with company_salesamt as (
	select 
	c.id cid,
	sum(s2.linetotal) salesamt, 
	extract(year from s.orderdate) "year",
	extract (quarter from s.orderdate) quarter_yr,
	p2.pcid categoryid
	from company c 
	join customer c2 on c2.companyname = c.cname
	join salesorderheader s on c2.customerid = s.customerid
	join salesorderdetail s2 on s.salesorderid = s2.salesorderid 
	join product2 p2 on s2.productid = p2.productid
	where date_part('y', s.orderdate) in (2012, 2013)
	group by "year", quarter_yr, c.id, p2.pcid),
cs_v as (
	select cs.*, cs."year" || '.' || cs.quarter_yr as qr, ca.cls as ccls
	from company_salesamt cs
	join company_abc ca on (cs.cid = ca.cid and ca."year"=cs."year"))
insert into company_sales
select cid, salesamt, "year", quarter_yr, qr, categoryid::int, ccls from cs_v;
```